In [70]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
from tensorflow.keras import layers

In [124]:
df = pd.read_csv('color_pedigree.csv')

In [125]:
df.head()

,base_color_male,with_white_male,base_color_female,with_white_female,prediction_color
0,black,1,black,0,black
1,black,0,black,1,black
2,black,1,black,1,black
3,black,0,black,0,black
4,blue,0,blue,0,blue


In [126]:
train, test = train_test_split(df, test_size=0.2)

# Input Pipeline

In [127]:
# EXERCISE: A utility method to create a tf.data dataset from a Pandas Dataframe.

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    
    # Use Pandas dataframe's pop method to get the list of targets.
    labels = dataframe.pop('prediction_color')
    
    # Create a tf.data.Dataset from the dataframe and labels.
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    
    if shuffle:
        # Shuffle dataset.
        ds = ds.shuffle(buffer_size=len(dataframe))
        
    # Batch dataset with specified batch_size parameter.
    ds = ds.batch(batch_size)
    
    return ds

In [128]:
batch_size = 4 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [129]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of base color female:', feature_batch['base_color_female'])
    print('A batch of preds color:', label_batch )

Every feature: ['base_color_male', 'with_white_male', 'base_color_female', 'with_white_female']
A batch of base color female: tf.Tensor([b'black' b'blue' b'creme' b'creme'], shape=(4,), dtype=string)
A batch of preds color: tf.Tensor([b'black tortie' b'black tortie' b'red' b'blue tortie'], shape=(4,), dtype=string)


In [131]:
example_batch = next(iter(train_ds))[0]
# A utility method to create a feature column and to transform a batch of data.
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column, dtype='float64')
    print(feature_layer(example_batch).numpy())

In [132]:
# EXERCISE: Fill in the missing code below
feature_columns = []


numeric_columns =['with_white_male', 'with_white_female']

for header in numeric_columns:
    # Create a numeric feature column  out of the header.
    numeric_feature_column = feature_column.numeric_column(header)
    
    feature_columns.append(numeric_feature_column)



# Indicator Cols.
# Create a categorical vocabulary column out of the categories
# ['fixed', 'normal', 'reversible'] with the key specified as 'thal'.
base_color_female = feature_column.categorical_column_with_vocabulary_list('base_color_female', ['black', 'blue', 'red', 'creme', 'black tortie', 'blue tortie'])

# Create an indicator column out of the created thal categorical column
base_color_female_one_hot = feature_column.indicator_column(base_color_female)

feature_columns.append(base_color_female_one_hot)


# Indicator Cols.
# Create a categorical vocabulary column out of the categories
# ['fixed', 'normal', 'reversible'] with the key specified as 'thal'.
base_color_male = feature_column.categorical_column_with_vocabulary_list('base_color_male', ['black', 'blue', 'red', 'creme'])

# Create an indicator column out of the created thal categorical column
base_color_male_one_hot = feature_column.indicator_column(base_color_male)

feature_columns.append(base_color_male_one_hot)



In [133]:
# EXERCISE: Create a Keras DenseFeatures layer and pass the feature_columns you just created.
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [135]:
batch_size = 4
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [136]:
model = tf.keras.Sequential([
        feature_layer,
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=test_ds,
          epochs=100)

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "C:\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "C:\Anaconda3\lib\asyncio\base_events.py", line 1890, in _run_once
      hand

# Experiment Without Pipeline

In [137]:
df = pd.read_csv('color_pedigree.csv')

In [138]:
df.head()

,base_color_male,with_white_male,base_color_female,with_white_female,prediction_color
0,black,1,black,0,black
1,black,0,black,1,black
2,black,1,black,1,black
3,black,0,black,0,black
4,blue,0,blue,0,blue


In [139]:
X = df.drop('prediction_color', axis=1)
y = df['prediction_color']

In [140]:
X

,base_color_male,with_white_male,base_color_female,with_white_female
0,black,1,black,0
1,black,0,black,1
2,black,1,black,1
3,black,0,black,0
4,blue,0,blue,0
...,...,...,...,...
123,creme,0,red,1
124,creme,0,creme,1
125,creme,0,black tortie,1
126,creme,0,black tortie,1


In [141]:
X = pd.get_dummies(X)

In [142]:
X

,with_white_male,with_white_female,base_color_male_black,base_color_male_blue,base_color_male_creme,base_color_male_red,base_color_female_black,base_color_female_black tortie,base_color_female_blue,base_color_female_blue tortie,base_color_female_creme,base_color_female_red
0,1,0,1,0,0,0,1,0,0,0,0,0
1,0,1,1,0,0,0,1,0,0,0,0,0
2,1,1,1,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,1,0,0,1,0,0,0,0,0,0,1
124,0,1,0,0,1,0,0,0,0,0,1,0
125,0,1,0,0,1,0,0,1,0,0,0,0
126,0,1,0,0,1,0,0,1,0,0,0,0


In [143]:
y

0             black
1             black
2             black
3             black
4              blue
           ...     
123             red
124           creme
125    black tortie
126             red
127     blue tortie
Name: prediction_color, Length: 128, dtype: object

In [144]:
X.shape

(128, 12)

In [148]:
y = pd.get_dummies(y)

In [149]:
y

,black,black tortie,blue,blue tortie,creme,red
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
123,0,0,0,0,0,1
124,0,0,0,0,1,0
125,0,1,0,0,0,0
126,0,0,0,0,0,1


In [150]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, input_dim=12, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X,y,epochs=100)

Epoch 1/100
4/4 [==============================] - 3s 30ms/step - loss: 1.7488 - accuracy: 0.2734
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 1.6466 - accuracy: 0.4219
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 1.5604 - accuracy: 0.4141
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 1.4842 - accuracy: 0.3594
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.4157 - accuracy: 0.3672
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.3486 - accuracy: 0.3750
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 1.2765 - accuracy: 0.4297
Epoch 8/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2075 - accuracy: 0.4922
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1277 - accuracy: 0.6172
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.0510 - accuracy: 0.6797
Epoch 11/100
4/4 [==========

4/4 [==============================] - 0s 2ms/step - loss: 0.2897 - accuracy: 0.7891
Epoch 85/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2901 - accuracy: 0.8047
Epoch 86/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2840 - accuracy: 0.8125
Epoch 87/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2819 - accuracy: 0.8281
Epoch 88/100
4/4 [==============================] - 0s 2ms/step - loss: 0.2830 - accuracy: 0.7812
Epoch 89/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2858 - accuracy: 0.7734
Epoch 90/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2823 - accuracy: 0.7891
Epoch 91/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2861 - accuracy: 0.7422
Epoch 92/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2836 - accuracy: 0.7734
Epoch 93/100
4/4 [==============================] - 0s 3ms/step - loss: 0.2835 - accuracy: 0.7891
Epoch 94/100
4/4 [===============